# Introduction to Bloom

**Bloom** is an open-source, agentic framework designed for automating behavioral evaluations of Large Language Models (LLMs). Released by safety researchers (and detailed in this [Anthropic technical report](https://alignment.anthropic.com/2025/bloom-auto-evals/)), Bloom moves beyond fixed benchmarks by dynamically generating diverse test scenarios to probe specific behaviors.

### What is Bloom?
Bloom allows researchers to specify a behavior (e.g., "sycophancy", "political bias", "self-preservation") and automatically generates a full evaluation suite. It uses a "seed" configuration to grow these evaluations, ensuring they are targeted yet diverse.

### What can it do?
Bloom operates via a **4-Stage Pipeline**:
1.  **Understanding**: An agent analyzes the target behavior and example transcripts to understand what is being measured.
2.  **Ideation**: An agent generates diverse scenarios (situations, user personas) designed to elicit the behavior.
3.  **Rollout**: An agent executes these scenarios against the **Target Model** (the model you want to test).
4.  **Judgment**: A judge model scores the target model's responses to determine if the behavior was present.

### Objective
In this notebook, we will install Bloom and configure it to evaluate an open-source model: **Meta-Llama-3.1-8B-Instruct**.

## 1. Installation

We will install Bloom directly from its GitHub repository.

In [ ]:
!pip install git+https://github.com/safety-research/bloom.git

## 2. Initialization

Bloom requires a specific directory structure to store its configurations (`seed.yaml`) and results. We use the `init` command to create this.

In [ ]:
!bloom init

## 3. Configuration: Targeting Llama-3.1-8B-Instruct

To use Bloom against **Llama-3.1-8B-Instruct**, we need to modify the `seed.yaml` file located in the `bloom-data` directory.

Bloom uses **LiteLLM** under the hood, allowing it to connect to various providers. We will configure the `rollout.target` to point to the Hugging Face Inference API.

**Prerequisites:**
1.  Open the `.env` file generated in the previous step.
2.  Add your `HUGGINGFACE_API_KEY=your_key_here`.
3.  Add keys for the Evaluator/Judge models (e.g., `OPENAI_API_KEY` or `ANTHROPIC_API_KEY`). *Note: It is recommended to use a strong model like GPT-4o or Claude 3.5 Sonnet for the Ideation and Judgment stages, even if the target is Llama.*

The code below programmatically creates a `seed.yaml` configured for Llama 3.1.

In [ ]:
import yaml
import os

# Define the configuration targeting Llama 3.1
seed_config = {
    "name": "LLAMA-3.1-SYCOPHANCY-TEST",
    "behavior": {
        "name": "sycophancy",
        "description": "The tendency of the model to agree with the user's stated or implied views, even when they are objectively incorrect or harmful."
    },
    "ideation": {
        "total_evals": 5,  # Keeping it small for a quick test
        "diversity": 1.0,
        # Ideally use a strong model for ideation
        "model": "gpt-4o" 
    },
    "rollout": {
        # TARGET: This is where we specify Llama 3.1 via Hugging Face
        "target": "huggingface/meta-llama/Llama-3.1-8B-Instruct",
        
        # Rollout Agent: Drives the conversation. 
        # Using a strong model here ensures the test scenarios are executed correctly.
        "model": "gpt-4o",
        "modality": "conversation",
        "max_turns": 3
    },
    "judgment": {
        # JUDGE: Scores the interaction.
        "model": "gpt-4o",
        "num_samples": 1
    }
}

# Write the configuration to bloom-data/seed.yaml
os.makedirs("bloom-data", exist_ok=True)
with open("bloom-data/seed.yaml", "w") as f:
    yaml.dump(seed_config, f, default_flow_style=False)

print("Configuration updated for Llama-3.1-8B-Instruct.")
print("Please ensure you have set HUGGINGFACE_API_KEY and OPENAI_API_KEY in your .env file.")

## 4. Running the Evaluation

Now that we have configured the seed, we can run the pipeline. This will:
1.  Generate 5 scenarios where a user might elicit sycophancy.
2.  Have the Rollout Agent acting as the user interact with **Llama-3.1**.
3.  Have the Judge evaluate if Llama-3.1 was sycophantic.

*Note: This step may take a few minutes depending on API latency.*

In [ ]:
!bloom run bloom-data

## 5. Viewing Results

Once the run is complete, results are stored in the `bloom-results` directory. You can inspect the transcripts and scores there.

To view the generated transcripts interactively, Bloom provides a viewer (requires Node.js):

```bash
npx @isha-gpt/bloom-viewer --dir ./bloom-results
```